# BONUS Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

Utilizando un conjunto de datos que hemos compartido, se busca evaluar si existen diferencias
significativas en el número de vuelos reservados según el nivel educativo de los clientes. Para ello, los
pasos que deberas seguir son:

- Preparación de Datos: Filtra el conjunto de datos para incluir únicamente las columnas relevantes:
'Flights Booked' y 'Education'.
- Análisis Descriptivo: Agrupa los datos por nivel educativo y calcula estadísticas descriptivas
básicas (como el promedio, la desviación estandar) del número de vuelos reservados para cada
grupo.
- Prueba Estadística: Realiza una prueba de hipótesis para determinar si existe una diferencia
significativa en el número de vuelos reservados entre los diferentes niveles educativos.

In [14]:
# importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest
from itertools import combinations


# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#from scipy.stats import shapiro, kstest, poisson, chisquare, ttest_ind, levene, bartlett, sem, ppf
import scipy.stats as stats

1. Preparación de datos para la filtración del conjunto. 

In [53]:
# Cargamos los datos combinados 

df = pd.read_csv('df_combinado.csv', index_col=0)
df.head(5)

,año,mes,vuelos_reservados,vuelos_con_acompañantes,vuelos_totales,distancia,puntos_acumulados,puntos_canjeados,costo_en_dolares_de_puntos_canjeados,provincia,ciudad,codigo_postal,genero,educacion,salario,estado_civil,tarjeta_lealtad,valoracion_cliente,tipo_inscripcion,año_inscripcion,mes_inscripcion,año_cancelacion,mes_cancelacion
numero_fidelizacion,,,,,,,,,,,,,,,,,,,,,,,
100018,2017,1,3,0,3,1521,152.0,0,0,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,0,0
100102,2017,1,10,4,14,2030,203.0,0,0,Ontario,Toronto,M1R 4K3,Male,College,73455.0,Single,Nova,2887.74,Standard,2013,3,0,0
100140,2017,1,6,0,6,1200,120.0,0,0,British Columbia,Dawson Creek,U5I 4F1,Female,College,73455.0,Divorced,Nova,2838.07,Standard,2016,7,0,0
100214,2017,1,0,0,0,0,0.0,0,0,British Columbia,Vancouver,V5R 1W3,Male,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,0,0
100272,2017,1,0,0,0,0,0.0,0,0,Ontario,Toronto,P1L 8X8,Female,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,0,0


In [54]:
# 1. Preparación de Datos
df_filtrado = df[['vuelos_reservados', 'educacion']]
df_filtrado

,vuelos_reservados,educacion
numero_fidelizacion,,
100018,3,Bachelor
100102,10,College
100140,6,College
100214,0,Bachelor
100272,0,Bachelor
...,...,...
999902,0,College
999911,0,Doctor
999940,3,Bachelor


2. Análisis descriptivo 

In [39]:
datos_agrupados = df_filtrado.groupby('educacion')['vuelos_reservados'].describe()
print(datos_agrupados)

                         count      mean       std  min  25%  50%  75%   max
educacion                                                                   
Bachelor              252567.0  4.110288  5.221671  0.0  0.0  1.0  8.0  21.0
College               102260.0  4.169744  5.246040  0.0  0.0  1.0  8.0  21.0
Doctor                 17731.0  4.175512  5.256971  0.0  0.0  1.0  8.0  21.0
High School or Below   18915.0  4.176209  5.239267  0.0  0.0  1.0  8.0  21.0
Master                 12287.0  4.200700  5.213956  0.0  0.0  1.0  8.0  21.0


3. Prueba Estadística (ANOVA)

Suponiendo que 'educacion' es una variable categórica y 'vuelos_reservados' es una variable numérica

- Se utiliza una prueba ANOVA (análisis de varianza) para comparar las medias de los diferentes grupos. La hipótesis nula es que las medias de todos los grupos son iguales. 
Si el valor p es menor que el nivel de significancia (α), se rechaza la hipótesis nula y se concluye que existe al menos un grupo con una media diferente.

In [47]:
# Creamos una lista de los grupos (niveles educativos)

grupos = df_filtrado['educacion'].unique()
grupos

array(['Bachelor', 'College', 'Master', 'High School or Below', 'Doctor'],
      dtype=object)

In [48]:
# Creamos un diccionario para almacenar los datos de cada grupo

datos = {group: df_filtrado[df_filtrado['educacion'] == group]['vuelos_reservados'].values for group in grupos}
datos

{'Bachelor': array([3, 0, 0, ..., 0, 3, 0]),
 'College': array([10,  6,  0, ...,  1,  0,  0]),
 'Master': array([ 0, 11,  9, ...,  0,  6,  0]),
 'High School or Below': array([ 0,  0,  6, ...,  0, 16,  3]),
 'Doctor': array([11,  0,  0, ...,  0, 15,  0])}

In [51]:
# Realizamos la prueba ANOVA

F_statistic, p_value = stats.f_oneway(*datos.values())

# F_statistic: Mide la diferencia entre las varianzas de los grupos.
# p_value: Nos indica la probabilidad de que la diferencia observada sea debida al azar.

F_statistic, p_value

(3.57895773849834, 0.00635298173601954)

Al combinar el estadístico F y el valor p, podemos determinar si existen diferencias significativas entre los grupos que estamos comparando. 
Es importante recordar que estos valores deben interpretarse en el contexto de la pregunta de investigación y del diseño del estudio.

In [52]:
# Interpretamos los resultados

alpha = 0.05
if p_value < alpha:
    print("Existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.")
else:
    print("No se encontró una diferencia significativa.")

Existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.
